In [1]:
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, holdout_split, year_split, births_per_year
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import kendalltau
import shap
import matplotlib
import warnings
warnings.filterwarnings('ignore')

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


### Load data

In [2]:
dfraw = load_names_from_web(category='national', hide_pre_1937=True, use_existing_files=True)
# traintestval, holdout = year_split(dfraw, holdout_size=30)
trainval, test = year_split(dfraw, holdout_size=30)
births_by_year = births_per_year()

In [3]:
# function to get all names known to the model
def get_all_known_names(data):

    names = data.groupby(['state', 'name', 'M/F']).size().reset_index()
    names = names[['state', 'name', 'M/F']]
    return names

In [4]:
# function to select only the names that meet a certain popularity threshold
def select_top_names(data, first_year_to_predict, cutoff=10):
    '''
    Select names that have had at least one year at or above the cutoff count?
    Could also try to redo this to do the cutoff for the most recent known year's data
    '''

    data_noleak = data[data['year'] < first_year_to_predict]
    names = data_noleak[data_noleak['count'] >= cutoff][['state', 'name', 'M/F']].drop_duplicates()
    # display(names)
    data = names.merge(data, how='left', on=['state', 'name', 'M/F'])
    # display(data)

    return data

### Classify whether a name has had longevity already, given the data we know

In [5]:
def classify(df, names_csv):

    '''
    PEAKS
    '''

    # peaks = df.loc[df.groupby(['state', 'name', 'M/F'])['count'].idxmax()][['state', 'name', 'M/F', 'year']]
    # peaks = peaks.rename(columns={'year': 'peak_year'})
    # df = df.merge(peaks, how='left', on=['state', 'name', 'M/F'])

    df = df.sort_values(by=['year', 'state', 'name', 'M/F'])
    all_names = df.drop(columns='count').drop_duplicates(subset=['state', 'name', 'M/F']).rename(columns={'year': 'first_year'})
    # display(all_names)
    all_years = range(1937, 2023)
    all_names = all_names.assign(key=1)
    all_years = pd.DataFrame({'year': all_years, 'key': 1})
    all_names_all_years = pd.merge(all_names, all_years, on='key').drop(columns='key')
    df = all_names_all_years.merge(df, how='left', on=['state', 'name', 'M/F', 'year'])
    df['count'] = df['count'].fillna(0)
    df = df[df['year'] >= df['first_year']]

    df['pct'] = df.apply(lambda row: row['count'] / births_by_year.at[row['year'], row['M/F']], axis=1)

    peaks = df.loc[df.groupby(['state', 'name', 'M/F'])['pct'].idxmax()][['state', 'name', 'M/F', 'year', 'pct']]
    peaks = peaks.rename(columns={'year': 'peak_pct_year', 'pct': 'peak_pct'})
    df = df.merge(peaks, how='left', on=['state', 'name', 'M/F'])

    df['has_peaked'] = df.apply(lambda row: 1 if row['year'] >= row['peak_pct_year'] else 0, axis=1)

    '''
    LONGEVITY
    '''

    names = pd.read_csv(names_csv)
    names = names[['state', 'name', 'M/F']]
    names['longevity'] = 1
    # display(names)

    df = df.merge(names, how='left', on=['state', 'name', 'M/F'])
    df['longevity'] = df['longevity'].fillna(0)
    # display(df)

    return df

# classify(trainval, 'longevity_names_1992.csv')

### Evaluation function: score y_pred vs. y_true

In [6]:
def evaluate(predictor, data_held_out, first_year_to_predict, metric='accuracy'):

    # display('data_held_out:')
    # display(data_held_out)

    # most_recent_year = data_held_out['year'].max()
    most_recent_year = first_year_to_predict - 1
    
    # years_to_predict = range(first_year_to_predict, most_recent_year+1)

    # only allow the model to see data from before the year to predict
    historical_data = data_held_out[data_held_out['year'] < first_year_to_predict]

    # display('historical_data:')
    # display(historical_data)

    # get our model's predictions
    predictions = predictor.predict(historical_data)

    all_known_names = get_all_known_names(historical_data)

    # now we need to know the "answers" -- what the real classes are, based on information in the "future"
    data_held_out = classify(data_held_out, 'Data/longevity_names_2022.csv')
    data_held_out = data_held_out[data_held_out['year'] == most_recent_year]
    # print(f'Predictions for {year_to_predict}:')

    # names_to_predict = all_known_names.copy()
    # names_to_predict['year'] = year_to_predict
    # display(names_to_predict)

    observed = all_known_names.merge(data_held_out, how='left', on=['state', 'name', 'M/F'])

    observed['y'] = observed['longevity'].fillna(0)
    # display(observed)

    score_df = observed.merge(predictions, how='left', on=['state', 'name', 'M/F'], suffixes=('_true', '_pred'))
    # display(score_df)

    y_true = score_df['y_true']
    y_pred = score_df['y_pred']

    if metric == 'accuracy':
        score = accuracy_score(y_true, y_pred)
        print(f'Score: {score}')

    if metric == 'f1':
        score = f1_score(y_true, y_pred)
        print(f'Score: {score}')

    if metric == 'all':
        print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
        print(f'Precision: {precision_score(y_true, y_pred)}')
        print(f'Recall: {recall_score(y_true, y_pred)}')
        print(f'F1: {f1_score(y_true, y_pred)}')
    
    return score_df[(score_df['y_true'] == 1) | (score_df['y_true'] != score_df['y_pred'])]

### Create dummy classifier (assumes name has longevity if it's had longevity as defined by the data known to the model)

In [7]:
class DummyClassifier():

    def __init__(self, strategy='constant'):
        self.strategy = strategy
    
    def predict(self, historical_data):

        all_known_names = get_all_known_names(historical_data)

        most_recent_year = historical_data['year'].max()

        predictions = all_known_names.copy()

        if self.strategy == 'constant':
            predictions['y'] = 0
        elif self.strategy == 'classify':
            historical_data = classify(historical_data, 'Data/longevity_names_1992.csv')
            most_recent_year_data = historical_data[historical_data['year'] == most_recent_year].drop(columns=['year'])
            predictions = predictions.merge(most_recent_year_data, how='left', on=['state', 'name', 'M/F'])
            predictions['longevity'] = predictions['longevity'].fillna(0)
            predictions['y'] = predictions['longevity']

        return predictions

### Get score for dummy classifier

In [8]:
first_year_to_predict = 1993
cutoff = 100
# data_to_fit = select_top_names(trainval, first_year_to_predict=first_year_to_predict, cutoff=0)
data_to_eval = select_top_names(test, first_year_to_predict=first_year_to_predict, cutoff=cutoff)
results = evaluate(predictor=DummyClassifier(strategy='classify'), data_held_out=data_to_eval, first_year_to_predict=first_year_to_predict, metric='all')

Accuracy: 0.9807953004970628
Precision: 0.9387755102040817
Recall: 0.359375
F1: 0.519774011299435


### Load actor and character name data

In [9]:
def load_movie_data():

    movies = pd.read_csv('Data/cleaned_cast_and_characters.csv')
    movies['release_year'] = movies['release_date'].fillna('0000').str[:4].astype(int)
    return movies

movies = load_movie_data()

### Create our prediction model

In [10]:
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier

class MyPredictor():

    def __init__(self):
        
        # params
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff',] # best overall?
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'first_letter_1_pct', 'first_letter_2_pct', 'first_letter_3_pct'] # first letter pct change helps in some years
        # cols_to_keep = ['this_year', 'M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'thisyear_count_opp']
        self.cols_to_keep = ['M/F', 'sum', 'median_age', 'thisyear_count', 'diff', 'thisyear_count_opp', 'diff2', 'shift', 'pct_change', 'accel', 'after_peak', 'years_since_peak', 'first_letter_1_pct', 'first_letter_2_pct', 'first_letter_3_pct', 'actor_recency_score', 'character_recency_score', 'longevity']
        # self.cols_to_keep = ['years_since_peak']
        # self.cols_to_keep = ['M/F', 'median_age', 'thisyear_count', 'diff', 'thisyear_count_opp', 'diff2', 'shift', 'pct_change', 'accel', 'after_peak', 'years_since_peak', 'first_letter_1_pct', 'first_letter_2_pct', 'first_letter_3_pct', 'actor_recency_score', 'character_recency_score']
        categorical_features = ['M/F', 'after_peak']
        max_leaf_nodes = 31 # 16
        max_iter = 200 # 100
        loss = 'log_loss' # abs better than default

        categorical_features = [True if f in categorical_features else False for f in self.cols_to_keep]
        # print(categorical_features)
        
        self.pipe = make_pipeline(
            ColumnTransformer(
                transformers=[
                    # ('category_encoder', LabelEncoder(), categorical_features),
                    ('cols_to_keep', 'passthrough', self.cols_to_keep),
                ], remainder='drop'),
            HistGradientBoostingClassifier(
                random_state=0,
                categorical_features=categorical_features,
                max_leaf_nodes=max_leaf_nodes,
                max_iter=max_iter,
                loss=loss
            )
        )

        self.gender_encoding = {'M': 0, 'F': 1}

        self.explainer = None

    def preprocess(self, df, latest_known_year, names_csv):

        # find median age of people with name, 
        # total born with that name,
        # and latest year's count

        df = df.copy()

        df = classify(df, names_csv)
        df['after_peak'] = df['peak_pct_year'].apply(lambda x: 1 if x < latest_known_year else 0)

        df = df.sort_values(by='year')
        df['cumsum'] = df.groupby(['state', 'name', 'M/F'])['count'].cumsum()
        df['sum'] = df.groupby(['state', 'name', 'M/F'])['count'].transform('sum')

        df['diff'] = df.groupby(['state', 'name', 'M/F'])['count'].diff()
        df['diff2'] = df.groupby(['state', 'name', 'M/F'])['count'].diff(2)
        df['shift'] = df.groupby(['state', 'name', 'M/F'])['count'].shift()
        df['pct_change'] = df.groupby(['state', 'name', 'M/F'])['count'].pct_change()
        df['accel'] = df.groupby(['state', 'name', 'M/F'])['diff'].diff()

        percentage_of_total_per_year = {}
        percentage_change_per_year = {}

        def first_letters(df, n, percentage_of_total_per_year, percentage_change_per_year):
            df['first_letter_'+str(n)] = df['name'].str[0:n].str.lower()
            total_names_per_year = df.groupby(['year', 'state', 'M/F'])['count'].sum()
            letter_names_per_year = df.groupby(['year', 'state', 'M/F', 'first_letter_'+str(n)])['count'].sum()
            percentage_of_total_per_year[n] = (letter_names_per_year / total_names_per_year).rename('first_letter_'+str(n)+'_pct')
            # display(percentage_of_total_per_year)
            percentage_change_per_year[n] = percentage_of_total_per_year[n].groupby(['state', 'M/F', 'first_letter_'+str(n)]).pct_change().rename('first_letter_'+str(n)+'_pct_change')
            # display(percentage_change_per_year)
            return df
        
        for n in range(1, 4):
            df = first_letters(df, n, percentage_of_total_per_year, percentage_change_per_year)
        
        # display(df[(df['name'] == 'Maximus')])

        medians = df[df['cumsum'] >= df['sum']/2]
        medians = medians.drop_duplicates(subset=['state', 'name', 'M/F'], keep='first')
        medians['median_age'] = latest_known_year - medians['year']
        # display(medians[medians['name'] == 'Madison'])

        medians = medians.drop(['count', 'cumsum', 'diff', 'shift', 'pct_change', 'accel', 'diff2', 'after_peak'], axis=1)

        thisyear = df[df['year'] == latest_known_year][['state', 'name', 'M/F', 'count', 'diff', 'shift', 'pct_change', 'accel', 'diff2', 'after_peak']].rename(columns={'count': 'thisyear_count'})
        # thisyear = thisyear.merge(percentage_of_total_per_year, how='left', on=['year', 'state', 'M/F', 'first_letter_1'])
        # thisyear = thisyear.merge(percentage_change_per_year, how='left', on=['year', 'state', 'M/F', 'first_letter_1'])
        # thisyear = thisyear.rename(columns={'year': 'this_year'})

        sex_counts = thisyear.groupby(['state', 'name', 'M/F'])['thisyear_count'].sum()
        thisyear_swapped = thisyear.copy()
        thisyear_swapped['M/F'] = thisyear_swapped['M/F'].replace({'M': 'F', 'F': 'M'})
        thisyear_swapped = thisyear_swapped.merge(sex_counts, how='left', on=['state', 'name', 'M/F'], suffixes=('', '_opp'))
        thisyear_swapped['M/F'] = thisyear_swapped['M/F'].replace({'M': 'F', 'F': 'M'})
        thisyear_swapped['thisyear_count_opp'] = thisyear_swapped['thisyear_count_opp'].fillna(0)
        thisyear = thisyear_swapped
        # display(thisyear)

        df = medians.merge(thisyear, how='left', on=['state', 'name', 'M/F']).rename(columns={'year': 'median_year'})
        # df2[['thisyear_count']] = df2[['thisyear_count']].fillna(0) # might want to shift this to 2 and fill in 2s for missing years? or maybe not

        # this is sort of a rough assumption that if a row didn't exist for this year, not only is the count 0, but so is the diff, shift and pct_change. not always true if the prior year had a count, but often true
        df[['thisyear_count', 'diff', 'shift', 'pct_change', 'accel', 'diff2']] = df[['thisyear_count', 'diff', 'shift', 'pct_change', 'accel', 'diff2']].fillna(0)

        df['after_peak'] = df['after_peak'].fillna(1)
        df['years_since_peak'] = latest_known_year - df['peak_pct_year']

        df['year'] = latest_known_year

        for n in range(1, 4):
            df = df.merge(percentage_of_total_per_year[n], how='left', on=['year', 'state', 'M/F', 'first_letter_'+str(n)])
            df = df.merge(percentage_change_per_year[n], how='left', on=['year', 'state', 'M/F', 'first_letter_'+str(n)])
            df['first_letter_'+str(n)+'_pct'] = df['first_letter_'+str(n)+'_pct'].fillna(0)
            df['first_letter_'+str(n)+'_pct_change'] = df['first_letter_'+str(n)+'_pct_change'].fillna(0)

        # display(df2)
        # display(df2.groupby(['state','name','M/F']).ngroups)

        # change M/F to 0/1 so it works with various models
        # (even HistGradientBoostingRegressor, which accepts categorical values,
        # still needs those values to be numbers not strings)
        df['M/F'] = df['M/F'].map(self.gender_encoding)

        # MOVIES
        known_movies = movies[movies['release_year'] <= latest_known_year] # avoid data leakage
        known_movies['years_since_release'] = latest_known_year - known_movies['release_year'] + 1
        known_movies['recency_score'] = 0.5 ** known_movies['years_since_release']
        actor_name_scores = known_movies.groupby('actor_first_name')['recency_score'].sum().reset_index().rename(columns={'actor_first_name': 'name', 'recency_score': 'actor_recency_score'})
        character_name_scores = known_movies.groupby('character_first_name')['recency_score'].sum().reset_index().rename(columns={'character_first_name': 'name', 'recency_score': 'character_recency_score'})
        df = df.merge(actor_name_scores, how='left', on='name')
        df = df.merge(character_name_scores, how='left', on='name')
        df['actor_recency_score'] = df['actor_recency_score'].fillna(0)
        df['character_recency_score'] = df['character_recency_score'].fillna(0)

        return df
    
    def fit(self, historical_data, first_year_to_predict, names_csv, years_to_fit=1, weight_decay=0.99):
        # first, we do need to find our y
        historical_data = classify(historical_data, names_csv)
        
        # first things first, we don't want to know about future data
        historical_data = historical_data[historical_data['year'] < first_year_to_predict]
        # at this point the data we don't want to know should be inaccessible

        # maybe there is a better way to do the following
        # but basically we just want the original columns for X and one new labeled column for y
        X_orig = historical_data[['state', 'year', 'name', 'M/F', 'count']]
        y_orig = historical_data[['state', 'year', 'name', 'M/F', 'longevity']]
        y_orig = y_orig.rename(columns={'longevity': 'y'})
        y_orig['M/F'] = y_orig['M/F'].map(self.gender_encoding)

        X_all = pd.DataFrame()
        y_all = pd.Series()

        # each year_to_fit is the year that's essentially our y for that loop
        for year_to_fit in range(first_year_to_predict - years_to_fit, first_year_to_predict):

            # now we "know" even less for X
            X = X_orig[X_orig['year'] <= year_to_fit]
            # y = historical_data[historical_data['year'] == year_to_fit]

            X = self.preprocess(X, latest_known_year=year_to_fit, names_csv='Data/longevity_names_1977.csv')
            # y = y[['state', 'name', 'M/F', 'count']].rename(columns={'count': 'y'})

            # display(X)

            data = X.merge(y_orig, how='left', on=['state', 'year', 'name', 'M/F'])
            data['y'] = data['y'].fillna(1) # if nothing for this name for this year, it has already peaked, so 1

            # assert data['y'].isna().any() == False

            # display(data)

            X = data.drop(columns=['y'])
            y = data['y']
            X['sample_weight'] = weight_decay ** (first_year_to_predict - year_to_fit)

            X_all = pd.concat([X_all, X], ignore_index=True)
            y_all = pd.concat([y_all, y], ignore_index=True)
        
        temp = X_all.copy()
        temp['y'] = y_all
        # display(temp)
        # display(temp.columns)

        sample_weights = X_all['sample_weight']
        X_all = X_all.drop(columns=['sample_weight'])

        self.pipe.fit(X_all, y_all, **{'histgradientboostingclassifier__sample_weight': sample_weights})
        # this seems like a silly way to pass params to individual steps of the pipeline, but it's true. See: https://stackoverflow.com/questions/36205850/sklearn-pipeline-applying-sample-weights-after-applying-a-polynomial-feature-t

        self.explainer = shap.Explainer(self.pipe.steps[-1][1], X_all.fillna(0)[self.cols_to_keep])

    def predict(self, historical_data):

        most_recent_year = historical_data['year'].max()

        df = self.preprocess(historical_data, latest_known_year=most_recent_year, names_csv='Data/longevity_names_1992.csv')
        # df = self.preprocess(historical_data, years_to_predict[0] - 1)

        df = df[df['year'] == most_recent_year]

        shap_values = self.explainer.shap_values(df.fillna(0)[self.cols_to_keep], check_additivity=False)
        # shap.summary_plot(shap_values, df.fillna(0)[self.cols_to_keep])
        shap_values = self.explainer(df.fillna(0)[self.cols_to_keep], check_additivity=False)
        shap.plots.bar(shap_values)
        shap.plots.beeswarm(shap_values)

        df['y'] = self.pipe.predict(df)

        # if we want to simply, do the following; 
        # but for now, might be useful to see all data displayed.
        # df = df[['state', 'year', 'name', 'M/F', 'y']]

        predictions = df.copy()

        # we have to reverse the mapping to send our predictions
        # (at least the way we currently have it set up)
        predictions['M/F'] = predictions['M/F'].map({v: k for k, v in self.gender_encoding.items()})

        # label the ones we know to be true
        predictions['y'] = predictions['y'].where(df['longevity'] != 1, 1)

        return predictions

### Use our prediction model, with SHAP analysis

100%|===================| 4404/4426 [00:20<00:00]        

Accuracy: 0.9807953004970628
Precision: 0.8771929824561403
Recall: 0.390625
F1: 0.5405405405405405


In [11]:
first_year_to_predict = 1993
years_to_predict_in_fitting = 15
years_to_fit_in_fitting = 30
name_usage_cutoff = 100
my_predictor = MyPredictor()
data_to_fit = select_top_names(trainval, first_year_to_predict=first_year_to_predict, cutoff=name_usage_cutoff)
data_to_eval = select_top_names(test, first_year_to_predict=first_year_to_predict, cutoff=name_usage_cutoff)
my_predictor.fit(historical_data=data_to_fit, first_year_to_predict=first_year_to_predict-years_to_predict_in_fitting, years_to_fit=years_to_fit_in_fitting, names_csv='Data/longevity_names_1992.csv')
results = evaluate(predictor=my_predictor, data_held_out=data_to_eval, first_year_to_predict=first_year_to_predict, metric='all')

In [12]:
# Uncomment any of these lines to more closely examine the correct or incorrect predictions made by the model

# results[['name', 'M/F', 'y_true', 'y_pred']]
# results[results['y_true'] == results['y_pred']][['name', 'M/F', 'y_true', 'y_pred']]
# results[(results['y_true'] == 1) & (results['y_pred'] == 0)][['name', 'M/F', 'y_true', 'y_pred']]
# results[(results['y_true'] == 0) & (results['y_pred'] == 1)][['name', 'M/F', 'y_true', 'y_pred']]